In [1]:
import os
from typing import TypedDict, Annotated, List
from dotenv import load_dotenv

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langchain_community.tools import GoogleSearchRun
from langchain_community.utilities import GoogleSearchAPIWrapper

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

c:\Users\anand\Desktop\perplexity\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
load_dotenv()

True

In [3]:
# Agent State
# ---------------------------------------------------------
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]

# ---------------------------------------------------------

In [4]:
# LLM Setup
# ---------------------------------------------------------
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.2,
    convert_system_message_to_human=True,
)

In [6]:
# Tools
# ---------------------------------------------------------
search_tool = GoogleSearchRun(api_wrapper=GoogleSearchAPIWrapper())
search_tool.name = "google_search"

C:\Users\anand\AppData\Local\Temp\ipykernel_81776\336742744.py:3: LangChainDeprecationWarning: The class `GoogleSearchRun` was deprecated in LangChain 0.0.33 and will be removed in 1.0. An updated version of the class exists in the `langchain-google-community package and should be used instead. To use it run `pip install -U `langchain-google-community` and import as `from `langchain_google_community import GoogleSearchRun``.
  search_tool = GoogleSearchRun(api_wrapper=GoogleSearchAPIWrapper())


In [7]:
tools = [search_tool]
llm_with_tools = llm.bind_tools(tools)

In [8]:
# Agent Node
# ---------------------------------------------------------
def agent_node(state: AgentState):
    system_prompt = SystemMessage("""
You are a helpful AI assistant.

- For casual chat, answer normally.
- For factual questions, news, or anything requiring updated information,
  CALL the `google_search` tool.
""")

    messages = [system_prompt] + state["messages"]
    ai_reply = llm_with_tools.invoke(messages)

    return {"messages": [ai_reply]}


In [9]:
# ---------------------------------------------------------
# Should Continue (Tool or End)
# ---------------------------------------------------------
def should_continue(state: AgentState):
    last = state["messages"][-1]
    return "tools" if last.tool_calls else "__end__"


In [ ]:
# ---------------------------------------------------------
# Create Graph
# ---------------------------------------------------------
def create_graph():
    graph = StateGraph(AgentState)

    graph.add_node("agent", agent_node)
    graph.add_node("tools", ToolNode(tools))

    graph.set_entry_point("agent")

    graph.add_conditional_edges(
        "agent",
        should_continue,
        {
            "tools": "tools",
            "__end__": END
        }
    )

    graph.add_edge("tools", "agent")

    app=graph.compile()
    response=app.invoke({"input":"hello"})
    print(response.content)
    return app  
